02で前処理をしたデータの読み込みと03で学習したモデルを使って推論を行うためのnotebookです。  

## 必要なライブラリのimport

In [50]:
import numpy as np
import pandas as pd
import os
import lightgbm as lgb
from sklearn.metrics import root_mean_squared_error
import pickle

In [51]:
# windows
if os.name == 'nt':
    path = '../../../data/processed/'
    modelpath = '../../src/models/nakagawa/'
else:
    if 'KAGGLE_DATA_PROXY_TOKEN' in os.environ.keys():
        path = '/kaggle/input/processed/'


# FIXME:
PATHを修正すること！！！

In [52]:
# FIXME: PATH
train_X_path = os.path.join(path,'processed20240618_train_X.csv')
test_X_path = os.path.join(path,'processed20240618_test_X.csv')
train_y_path = os.path.join(path,'processed20240618_train_y.csv')
test_y_path = os.path.join(path,'processed20240618_test_y.csv')
train_path = os.path.join(path,'processed20240618_train.csv')
test_path = os.path.join(path,'processed20240618_test.csv')

In [53]:
# データの読み込み
X_test = pd.read_csv(test_X_path)
y_test = pd.read_csv(test_y_path)
test = pd.read_csv(test_path)

del X_test['target']


## 保存されたモデルを使って予測を行う

In [54]:
def predict_model(test_X,model_name,test_y=False,flg=False):
    n_folds = 5
    predictions = np.zeros(len(test_X))
    features = [c for c in test_X.columns if c not in ['card_id', 'first_active_month']]

    for fold_ in range(n_folds):
        # モデルを読み込む
        with open(f'{modelpath}{model_name}_{fold_}.pkl', 'rb') as f:
            clf = pickle.load(f)

        # モデルで予測を行う
        fold_predictions = clf.predict(test_X[features])

        # 予測結果を足し合わせる
        predictions += fold_predictions

    # 平均を取る
    predictions /= n_folds
    if flg == False:
        # RMSEを計算する（テストデータに対して真の値がある場合）
        # y_testが存在する場合に限り
        score = root_mean_squared_error(test_y, predictions)
        print(f'RMSE={score}')
    return predictions

# テストデータを予測する 
## ①train_Xモデルを用いてtestデータを予測してRMSEを出す
## (モデルネーム確認すること！！)


In [55]:
print(predict_model(test_X=X_test,test_y=y_test,model_name='train_X_model_20240618'))

RMSE=3.683587536936083
[-0.77909236  0.119663   -0.16877793 ... -0.44612076  0.05208372
 -0.29031117]


## ②train_allモデルを用いてtestデータでの提出用の予測値を出す

### 提出用ファイルの作成　(モデルネーム確認すること！！)

In [56]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predict_model(test_X=test,model_name='train_all_model_20240618',flg=True)
sub_df.to_csv("submit.csv", index=False)